In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt#to plot accuracy
import cv2 as cv

import tensorflow as tf
from PIL import Image
import os
from keras.models import Model
from tensorflow import keras

from sklearn.model_selection import train_test_split #to split training and testing data
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import to_categorical #to convert the labels present in y_train and t_test into one-hot encoding
from keras.layers import Dense,  Dropout #to create CNN
%matplotlib inline

In [85]:
DATAPATH = r"TransferLearning/"

data = []
labels = []
classes = 4
cur_path = os.getcwd()
#Retrieving the images and their labels
for i in range(classes):
   path = os.path.join(DATAPATH,str(i))
   images = os.listdir(path)
   for a in images:
      try:
               image = Image.open(path + '/' + a)
               image = image.resize((30,30))
               image = np.array(image)
               image = image/255.0
               #sim = Image.fromarray(image)
               data.append(image)
               labels.append(i)
      except:
               print("Error loading image")
#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)

(451, 30, 30, 3) (451,)


In [86]:
#Splitting training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(360, 30, 30, 3) (91, 30, 30, 3) (360,) (91,)


In [87]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

In [122]:
image_generator = ImageDataGenerator(
                                        rotation_range=22,
                                        width_shift_range=0.15,
                                        height_shift_range=0.15,
                                        horizontal_flip=True,
                                        brightness_range=[0.2,1.0],
                                        zoom_range=[0.2,0.8],
                                        featurewise_center=True,
                                        featurewise_std_normalization=True,
                                        )

image_generator.fit(x_train)

In [123]:
from keras.models import load_model
model = load_model("traffic_classifier_prueba.h5")

In [129]:
for i in range(0,6):
    model.layers[i].trainable = False

In [130]:
for i in range(7,7):
    model.layers[i].trainable = True

In [131]:
addl = model.layers[8].output
addl = Dense(32)(addl)
addl = Dense(16,activation='relu')(addl)
addl = Dense(16,activation='relu')(addl)
adll = Dropout(0.25)(addl)
addl = Dense(8,activation='relu')(addl)
addl = Dense(8,activation='relu')(addl)
adll = Dropout(0.5)(addl)
addl = Dense(8,activation='relu')(addl)
addl = Dense(4,activation="softmax")(addl)

new_model = Model(inputs=model.input,outputs=addl)

In [132]:
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

new_model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12_input (InputLayer  [(None, 30, 30, 3)]      0         
 )                                                               
                                                                 
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        2432      
                                                                 
 conv2d_13 (Conv2D)          (None, 22, 22, 32)        25632     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 11, 11, 32)       0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 11, 11, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 9, 9, 64)          184

In [133]:
#train the model
with tf.device('/GPU:0'):
    epochs = 20
    history = new_model.fit(image_generator.flow(x_train, y_train, batch_size=2),
          steps_per_epoch=len(x_train) / 8, epochs=epochs)

Epoch 1/20
45/45 [==============================] - 1s 5ms/step - loss: 1.3767 - acc: 0.5778
Epoch 2/20
45/45 [==============================] - 0s 4ms/step - loss: 1.3674 - acc: 0.4778
Epoch 3/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3597 - acc: 0.4889
Epoch 4/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3536 - acc: 0.4778
Epoch 5/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3380 - acc: 0.5333
Epoch 6/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3172 - acc: 0.5556
Epoch 7/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3329 - acc: 0.4556
Epoch 8/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3195 - acc: 0.4778
Epoch 9/20
45/45 [==============================] - 0s 5ms/step - loss: 1.3311 - acc: 0.4222
Epoch 10/20
45/45 [==============================] - 0s 5ms/step - loss: 1.2877 - acc: 0.5222
Epoch 11/20
45/45 [==============================] - 0s 5ms/step - lo

In [134]:
new_model.save('transfer_learning_modelo_optimizado.h5')

TESTING DATA

In [135]:
model = load_model('transfer_learning_modelo_optimizado.h5')

In [136]:
import cv2 as cv
from statistics import mode

In [137]:
cropped = r"TransferLearning/Valid/1/9.png"
img = cv.imread(cropped)
#plt.imshow(img)

imgGray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
imgGray = cv.medianBlur(imgGray, 5)
print(imgGray.ndim)
img_copy = imgGray.copy()
rows = img_copy.shape[0]

circles = cv.HoughCircles(img_copy,cv.HOUGH_GRADIENT,1,rows/8, 
                         		param1 = 50, param2 = 45, 
                         		minRadius = 50, maxRadius = 150)

if circles is not None:		#Condicional - si detecta circulos
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cy = i[0]
            cyv = cy
            for j in circles[0, :]:
                cx = j[1]
                center = (cx, cyv) 	
                radius = j[2]
                minrec = (cx - radius, cyv - radius)
                maxrec = (cx + radius, cyv + radius)
            
                print(circles)
                #plt.imshow(frame)

                frame = (img[(cx-radius-15):(cx+radius+15),(cyv-radius-15):(cyv+radius+15)])
                #plt.imshow(img)


                frameNN = cv.resize(frame,(30,30))

                tests = []
                tests.append(np.array(frameNN))
                X = np.array(tests)

                pred = np.argmax(model.predict(X))
                #preds = []
                #preds = preds.append(pred)
                #print(preds)

                #predfinal = mode(preds)
                print("El número de la señal de tránsito es: ", pred)

                plt.imshow(frame)

2
